In [23]:
import os
import csv
import pandas as pd

In [2]:
df = pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\qld(new)\разделенный\Пестициды2019-0.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")

In [3]:
df1 = pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\qld(new)\разделенный\Пестициды2019-1.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")

In [24]:
df = pd.read_excel(r'C:\Users\Bauyrzhan\Desktop\колдауский\Пестициды.xlsx')

In [25]:
df1 = pd.read_excel(r'C:\Users\Bauyrzhan\Desktop\колдауский\Удобрения.xlsx')

In [30]:
df1 = df1[['Область', 'Дата подачи заявки', 'Наименование заявителя', 
        'Производитель удобрений', 'Наименование удобрения',
        'Объем удобрения', 'Единица измерения',
        'Площадь использования, га', 'Статус', 'Тип заявки',
        'Дата принятия заявки', 'Дата отрицательного решения',
        'Сумма субсидий (расчетная), тг.', 'Причина отрицательного решения',
        'Примечание к отрицательному решению', 'Группа', 'Год']]

In [31]:
kolonki = ['region','appDate','applicant',
 'provider','product','volume','unit',
 'areaOfUsage','status','typeOfApp','AppAcceptanceDate',
 'DateOfRejection','subsidies','reasonOfRejection',
 'NoteOnNegativeResolution','group','year']

In [32]:
df.columns = kolonki

In [33]:
df1.columns = kolonki

In [39]:
subsidies_02_raw = df

In [40]:
import csv
import pandas as pd
import pyodbc
import numpy as np
import matplotlib.pyplot as plt
server = '192.168.1.209'  #tcp:myserver.database.windows.net 
database = 'AA_DWH_X' #mydb
username = 'pbi_report' #myusername 
password = '1s22s22p6' #mypassword
# driver = r'C:\Users\M_MUKHANOVA\aads\driver\msodbcsql.msi'
conn = pyodbc.connect('DRIVER={sql server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()
print(conn)

In [41]:
qName_sql = ('SELECT * FROM qName')
qName = pd.read_sql(qName_sql, conn)
qName = qName.applymap(str)
qName.codERP = qName.codERP.str.pad(9, side='left', fillchar='0')

In [42]:
cato_sql = ('SELECT * FROM cato')
cato = pd.read_sql(cato_sql, conn)
cato = cato.applymap(str)

In [43]:
qWithProdCodes = pd.merge(subsidies_02_raw, qName[["name", "codERP"]], left_on = "product", right_on = "name", how='left').drop(columns = ["name"])
subsidies_02 = pd.merge(qWithProdCodes, cato[["name", "cato_Id"]], left_on = "region", right_on = "name", how='left').drop(columns = ["name"])

In [44]:
for i in range(len(subsidies_02['region'])):
    if subsidies_02['region'][i] == 'Шымкент г.а.':
        subsidies_02['cato_Id'][i] = '790000000'
    elif subsidies_02['region'][i] == 'Алматы г.а.':
        subsidies_02['cato_Id'][i] = '750000000'
    elif subsidies_02['region'][i] == 'Нур-Султан г.а.':
        subsidies_02['cato_Id'][i] = '710000000'

<ipython-input-44-4f0a292d31be>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subsidies_02['cato_Id'][i] = '790000000'


In [48]:
df['volume'] = df['volume'].apply(lambda x:x.replace(',','.') if type(x) == str else x)
df['volume'] = df['volume'].apply(lambda x:float(x.replace(' ','')) if type(x) == str else x)
df['col1'] = df.apply(lambda row: 1000*row['volume'] if row['unit'] == 'тонн' else row['volume'], axis=1)
df.drop(['volume'],axis=1,inplace=True)
df = df.rename(columns={"col1": "volume"})
df = df[['region','appDate','applicant','provider','product',
         'volume','unit','areaOfUsage','status','typeOfApp',
         'AppAcceptanceDate','DateOfRejection','subsidies',
         'reasonOfRejection','NoteOnNegativeResolution','group','year']]

In [ ]:
df.to_excel('pesticides.xlsx')
df.to_csv(r'C:\Users\Bauyrzhan\Desktop\колдауский\/' + 'pesticides' + '.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")

In [ ]:
df1.to_excel('fertilizers.xlsx')
df1.to_csv(r'C:\Users\Bauyrzhan\Desktop\колдауский\/' + 'fertilizers' + '.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")